# Data Transformation module

Imports

In [4]:
import numpy as np
from tqdm import tqdm

from data_access.insert.flux import insert_flux_trend_and_ma
from data_access.insert.flux_stat import insert_flux_stats
from data_access.read.flux import get_raw_flux_for_star_in_sector, get_all_stars_for_sector
from tools.flatten import add_wotan_trend
from tools.moving_average import add_moving_averages

### Variables Selection

In [5]:
sector = 5
wotan_window_length = 0.3
moving_average_window = 90

### Process

In [6]:
tics = get_all_stars_for_sector(sector)
for tic in tqdm(tics):
    flux = get_raw_flux_for_star_in_sector(sector, tic)
    add_wotan_trend(flux)
    add_moving_averages(flux, moving_average_window)
    insert_flux_trend_and_ma(flux)

    m_a_trend_flux = flux['m_a_trend_flux']
    mean_val = m_a_trend_flux.mean()
    max_val = m_a_trend_flux.max()
    min_val = m_a_trend_flux.min()
    std_val = m_a_trend_flux.std()

    # Calculate range
    range_val = max_val - min_val

    # Calculate amplitude
    sorted_fluxes = m_a_trend_flux.sort_values()
    top_5_percent = sorted_fluxes[int(0.95 * len(sorted_fluxes)):]
    bottom_5_percent = sorted_fluxes[:int(0.05 * len(sorted_fluxes))]
    amplitude_val = (np.median(top_5_percent) - np.median(bottom_5_percent)) / 2

    # Calculate beyond1std
    beyond1std_val = np.mean(np.abs(m_a_trend_flux - mean_val) > std_val)

    insert_flux_stats(sector, tic, mean_val, max_val, min_val, std_val, range_val, amplitude_val, beyond1std_val)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:30<00:00,  1.80s/it]
